In [ ]:
!pip install bs4

  Preparing metadata (setup.py) ... done
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1256 sha256=9a571b07ad5d1f7afe30993b3a2a75df6a0676f81baafbad20f58764b8008de4
  Stored in directory: /root/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
Successfully built bs4


In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import csv
import pandas as pd
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

In [ ]:
def get_data(url):
  #Tín code here
    session = requests.Session()
    retry = Retry(connect=3, backoff_factor=0.5)
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)

    html_text = session.get(url)
    soup = BeautifulSoup(html_text.content.decode('utf-8'), 'lxml')
    return soup

In [ ]:
#due to the structure of first is difference from other so we run first independently
#Create dict to store data
dict_data_first_page ={'question':[],
                      'answer':[]}
soup = get_data('https://tudu.com.vn/vn/khac/') #link first page
for url in soup.find_all('div', class_ = 'article-list-style-name'): #find class that have url question
    for a in url.find_all('a'):
        soup = get_data('https://tudu.com.vn' +a.get('href'))
        data_question = soup.find('div',id="content-request")
        data_answer = soup.find('div',id="content-reply")
        dict_data_first_page['question'].append(data_question.text)
        dict_data_first_page['answer'].append(data_answer.text)

dict_data_first_page

In [ ]:
#convert to df
df = pd.DataFrame(data =dict_data_first_page)
df

,question,answer
0,"Hỏi - 30/08/2023 Em chào mọi người ạ, cho em h...",Trả lời Chào em\nEm có thể nhờ người thân đến ...
1,Hỏi - 23/08/2023 Cho em hỏi bác sĩ có hẹn lấy ...,"Trả lời Chào bạn\nHiện tại, BV chưa triển khai..."
2,Hỏi - 27/08/2023 Em muốn biết chi phí tiến hàn...,"Trả lời Chào bạn\nHiện tại, BV chúng tôi chưa ..."
3,Hỏi - 26/08/2023 Em làm mất bảng kê khai chi p...,Trả lời Chào bạn\nNếu bạn muốn Bảng chi tiết k...
4,"Hỏi - 31/07/2023 Em chào anh/chị, em có một số...",Trả lời Chào bạn\nBạn có thể tham khảo thông t...
5,Hỏi - 08/07/2023 Bác sĩ cho em hỏi em có 1 bé ...,Trả lời Chào em\nEm nên đến khám tại khoa Chăm...
6,Hỏi - 10/07/2023 Em mới khám phụ khoa và bác s...,Trả lời Chào em\nMỗi người sẽ có đáp ứng với đ...
7,Hỏi - 11/07/2023 Bác sĩ ơi cho em hỏi ạvào ngà...,"Trả lời Chào bạn,\nQue thử thai là một xét ngh..."
8,Hỏi - 05/07/2023 Em đang mang thai được khoảng...,Trả lời Chào em\nEm có thể đến trực tiếp khoa ...
9,Hỏi - 10/07/2023 Kính gửi: Khoa Xét nghiệm di ...,"Trả lời Chào bạn\nVới trường hợp của bạn, bạn ..."


In [ ]:
def arr_data(base_url, so_trang):
  #Tín code here
  #this function just like cell first like about but it generalize for page 2 - end
    dict_data = {'question': [], 'answer': []}

    for i in range(2, so_trang + 1):
        soup = get_data(base_url + str(i) + '/')

        for url in soup.find_all('div', class_='article-list-style-name'):
            for a in url.find_all('a'):
                soup = get_data('https://tudu.com.vn' + a.get('href'))

                data_question = soup.find('div', id="content-request")
                data_answer = soup.find('div', id="content-reply")

                # Check if data_question and data_answer are not None before accessing their text attributes
                if data_question is not None:
                    dict_data['question'].append(data_question.text)
                else:
                    dict_data['question'].append(None)  # or any placeholder value

                if data_answer is not None:
                    dict_data['answer'].append(data_answer.text)
                else:
                    dict_data['answer'].append(None)  # or any placeholder value

    return dict_data


sức khỏe sinh sản 832 trang

sức khỏe của bé 473 trang

sức khỏe phụ nữ 283 trang

các biện pháp ngừa thai 156 trang

Hiếm muộn-vô sinh 248 trang

khác 67 trang

#https://tudu.com.vn/vn/khac/

#https://tudu.com.vn/vn/suc-khoe-mang-thai/

#https://tudu.com.vn/vn/suc-khoe-cua-be/

#https://tudu.com.vn/vn/suc-khoe-phu-nu/

#https://tudu.com.vn/vn/cac-bien-phap-ngua-thai/

#https://tudu.com.vn/vn/hiem-muon-vo-sinh/

In [ ]:
#từ dũ hospital has some kind of field that contain question such as:

#change the base_url variable with these link to crawl data from that link, and so trang should be under 200 (if not there will be some error connection)
base_url1 = 'https://tudu.com.vn/vn/khac/'
dict_data = arr_data(base_url1,67)

In [ ]:
df1 = pd.DataFrame(data =dict_data)
df1

,question,answer
0,Hỏi - 19/09/2022 Bác sỹ ơi cho em hỏi. Em bị h...,"Trả lời Chào bạn,\n MABPCOS là sản phẩm hỗ trợ..."
1,Hỏi - 24/05/2022 Dạ bác sĩ cho em thắc mắc xíu...,Trả lời Chào em\nXét nghiệm Hcg thì ăn uống bì...
2,Hỏi - 01/04/2022 Dạ chào các bác sĩ. Hôm 07/12...,Trả lời Chào bạn\nBạn đã từng khám khoa nào th...
3,Hỏi - 30/11/2021 Vấn đề đổi mới trứng thụ tinh...,"Trả lời Chào bạn Hậu, \n1/ Vấn đề đổi mới trứn..."
4,Hỏi - 08/01/2022 Chào Bác sĩ ạ!\r\nEm năm nay ...,"Trả lời Chào bạn,\nTrường hợp của bạn gọi là s..."
...,...,...
2352,None,None
2353,None,None
2354,None,None
2355,None,None


In [ ]:
#merge data
frame = [df,df1]
df_merged = pd.concat(frame,ignore_index=True)

In [ ]:
df_merged

,question,answer
0,Hỏi - 13/05/2023 Bác sĩ hẹn em ngày 7 chu kỳ k...,Trả lời Chào em\n\n\n\n\n\n\n\n\n\n\nThời điểm...
1,Hỏi - 08/05/2023 Dạ cho em hỏi lúc trước em đã...,Trả lời Chào em\nEm có thể đến khoa Chăm sóc t...
2,Hỏi - 21/04/2023 Chào Anh/Chị. Em muốn hỏi thủ...,Trả lời Chào bạn\nVợ chồng bạn phải đến Khoa H...
3,"Hỏi - 05/04/2023 Dạ, chào Bác sĩ! em tên Nguyễ...",Trả lời Chào bạn\nBạn có thể hỏi thông tin thê...
4,Hỏi - 14/03/2023 Bác sĩ cho hỏi chi phí hiện t...,Trả lời Chào bạn\nBạn có thể tham khảo các thô...
...,...,...
2362,None,None
2363,None,None
2364,None,None
2365,None,None


In [ ]:
df_merged.to_excel('/content/data_tudu_hiemmuon.xlsx')